In [1]:
pip install finance-datareader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.0 MB/s eta 0:00:00


In [2]:
pip install pykrx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 776.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 11.8 MB/s eta 0:00:00


In [3]:
import pandas as pd
import itertools
import numpy as np
import FinanceDataReader as fdr
from concurrent.futures import ThreadPoolExecutor
from pykrx import stock
from datetime import datetime

In [4]:
df = stock.get_market_ohlcv("20150721", "20250325", "005930")
df

,시가,고가,저가,종가,거래량,등락률
날짜,,,,,,
2015-07-21,25500,25540,24940,25260,194055,-0.941176
2015-07-22,24880,25200,24700,25060,268323,-0.791766
2015-07-23,24880,25060,24680,24680,208965,-1.516361
2015-07-24,24540,24760,24480,24580,196584,-0.405186
2015-07-27,24580,24940,24560,24600,243040,0.081367
...,...,...,...,...,...,...
2025-03-19,57400,59200,57400,58500,29421759,1.562500
2025-03-20,59200,60300,58500,60200,34989004,2.905983
2025-03-21,60900,61700,60400,61700,40155612,2.491694


In [5]:
df_filtered = df[df['등락률'] >= 5]  # 5% 이상 상승한 날 필터링


In [13]:
def get_performance(df, date, days):
    future_date = date + pd.DateOffset(days=days)  # 날짜 이동
    future_prices = df[df.index >= future_date]  # 이후 날짜의 데이터 찾기
    if not future_prices.empty:
        return (future_prices.iloc[0]['종가'] / df.loc[date]['종가']) - 1
    return None  # 날짜 초과 시 None 반환

# 결과 저장
df_filtered['T+5'] = [get_performance(df, date, 5) for date in df_filtered.index]
df_filtered['T+20'] = [get_performance(df, date, 20) for date in df_filtered.index]
df_filtered['T+30'] = [get_performance(df, date, 30) for date in df_filtered.index]

# 결과 확인
print(df_filtered[['등락률', 'T+5', 'T+20', 'T+30']])

                  등락률       T+5      T+20      T+30
날짜                                                 
2015-10-07   8.688097  0.007194  0.037570  0.069544
2020-03-20   5.704307  0.071586  0.081498  0.103524
2020-03-24  10.470588  0.019169  0.028754  0.061768
2020-06-03   6.031128  0.007339 -0.056881 -0.016514
2020-07-28   5.395683 -0.030717 -0.003413 -0.051195
2020-12-24   5.277402  0.006427  0.152956  0.149100
2021-01-08   7.117008  0.010135 -0.057432 -0.065315
2021-11-22   5.196629 -0.034713  0.025367  0.060080
2023-09-01   6.128550 -0.014085 -0.029577 -0.049296
2024-03-20   5.631868  0.016905  0.087126  0.009103
2024-11-15   7.214429  0.033645  0.003738  0.039252
2024-11-18   5.981308  0.021164 -0.058201 -0.031746
2025-03-17   5.301645  0.050347       NaN       NaN


<ipython-input-13-9423e160376a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['T+5'] = [get_performance(df, date, 5) for date in df_filtered.index]
<ipython-input-13-9423e160376a>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['T+20'] = [get_performance(df, date, 20) for date in df_filtered.index]
<ipython-input-13-9423e160376a>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [15]:
df_filtered[['T+5', 'T+20', 'T+30']].describe()

,T+5,T+20,T+30
count,13.000000,12.000000,12.000000
mean,0.012646,0.017625,0.023192
std,0.029453,0.065785,0.068075
min,-0.034713,-0.058201,-0.065315
25%,0.006427,-0.036403,-0.036133
50%,0.010135,0.014553,0.024178
75%,0.021164,0.048552,0.063712
max,0.071586,0.152956,0.149100
